In [1]:
from methods import *
import os, shutil

/Users/henryjin/Documents/Harvard CSE/Year 1/Fall 2021/NLP/WEDA/nlp_aug_w.py:195: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not '']
/Users/henryjin/Documents/Harvard CSE/Year 1/Fall 2021/NLP/WEDA/nlp_aug_w.py:303: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not '']


In [2]:
# We have percent_data/percentages/dataset/train_orig.txt

#load hyperparameters
sizes = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

folders_list =  ['percent_data/' + str(int(size*100)) for size in sizes]
#size_folders = ['size_data_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]


# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [3]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [12]:
#for each method
a_method = 'eda' #### TOGGLE EDA/WEDA

writer_cnn = open('outputs_percent/cnn_' + a_method + '_' + get_now_str() + '.txt', 'w')

#for storing the performances
performances_cnn = {size_folder:[] for size_folder in folders_list}

#for each percentage dataset
for size_folder in folders_list:

    writer_cnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]



    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        

        train_path = dataset_folder + '/train_eda_pct.txt' #### TOGGLE EDA/WEDA
        test_path = dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn aug ", acc, dataset_folder)

        performances_cnn[size_folder].append(acc)
        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn[alpha].append(acc)
        print("rnn aug", acc)
        """

writer_cnn.write(str(performances_cnn) + '\n')
#writer_rnn.write(str(performances_rnn)+"\n")
for size_folder in performances_cnn:
    line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
    writer_cnn.write(line + '\n')
    print(line)
"""
for alpha in performances_rnn:
    line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
    writer_rnn.write(line + '\n')
    print(line)
"""
print(performances_cnn)

writer_cnn.close()    


Epoch 1/20
1/1 [==============================] - 1s 937ms/step - loss: 0.6962 - accuracy: 0.4825 - val_loss: 0.6711 - val_accuracy: 0.7945
Epoch 2/20
1/1 [==============================] - 0s 149ms/step - loss: 0.6643 - accuracy: 0.8250 - val_loss: 0.6524 - val_accuracy: 0.8767
Epoch 3/20
1/1 [==============================] - 0s 162ms/step - loss: 0.6367 - accuracy: 0.8995 - val_loss: 0.6295 - val_accuracy: 0.9315
Epoch 4/20
1/1 [==============================] - 0s 167ms/step - loss: 0.6082 - accuracy: 0.9528 - val_loss: 0.6056 - val_accuracy: 0.9041
Epoch 5/20
1/1 [==============================] - 0s 160ms/step - loss: 0.5796 - accuracy: 0.9574 - val_loss: 0.5826 - val_accuracy: 0.9178
Epoch 6/20
1/1 [==============================] - 0s 154ms/step - loss: 0.5513 - accuracy: 0.9589 - val_loss: 0.5590 - val_accuracy: 0.9178
Epoch 7/20
1/1 [==============================] - 0s 158ms/step - loss: 0.5226 - accuracy: 0.9619 - val_loss: 0.5345 - val_accuracy: 0.9178
Epoch 8/20
1/1 [====

7/7 [==============================] - 1s 152ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0473 - val_accuracy: 0.9811
Epoch 19/20
7/7 [==============================] - 1s 142ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0449 - val_accuracy: 0.9838
Epoch 20/20
7/7 [==============================] - 1s 142ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0445 - val_accuracy: 0.9838
cnn aug  0.7998856489422527 percent_data/10/sst2
Epoch 1/20
13/13 [==============================] - 3s 181ms/step - loss: 0.6636 - accuracy: 0.6318 - val_loss: 0.5570 - val_accuracy: 0.7788
Epoch 2/20
13/13 [==============================] - 2s 155ms/step - loss: 0.5043 - accuracy: 0.8002 - val_loss: 0.4194 - val_accuracy: 0.8112
Epoch 3/20
13/13 [==============================] - 2s 155ms/step - loss: 0.3650 - accuracy: 0.8462 - val_loss: 0.3279 - val_accuracy: 0.8654
Epoch 4/20
13/13 [==============================] - 2s 155ms/step - loss: 0.2556 - accuracy: 0.9101 - val_loss: 0.

26/26 [==============================] - 4s 164ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0761 - val_accuracy: 0.9740
Epoch 15/20
26/26 [==============================] - 4s 152ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0753 - val_accuracy: 0.9733
Epoch 16/20
26/26 [==============================] - 4s 152ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0748 - val_accuracy: 0.9729
Epoch 17/20
26/26 [==============================] - 4s 152ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9740
Epoch 18/20
26/26 [==============================] - 4s 166ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0755 - val_accuracy: 0.9736
Epoch 19/20
26/26 [==============================] - 5s 175ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.0749 - val_accuracy: 0.9736
cnn aug  0.8256146369353916 percent_data/40/sst2
Epoch 1/20
33/33 [==============================] - 7s 188ms/step - loss: 0.6248 - accuracy: 0.6516 - val

52/52 [==============================] - 9s 171ms/step - loss: 0.0065 - accuracy: 0.9997 - val_loss: 0.0825 - val_accuracy: 0.9675
Epoch 9/20
52/52 [==============================] - 8s 159ms/step - loss: 0.0044 - accuracy: 0.9998 - val_loss: 0.0824 - val_accuracy: 0.9685
Epoch 10/20
52/52 [==============================] - 8s 161ms/step - loss: 0.0033 - accuracy: 0.9999 - val_loss: 0.0841 - val_accuracy: 0.9677
Epoch 11/20
52/52 [==============================] - 8s 153ms/step - loss: 0.0027 - accuracy: 0.9998 - val_loss: 0.0840 - val_accuracy: 0.9684
Epoch 12/20
52/52 [==============================] - 8s 154ms/step - loss: 0.0023 - accuracy: 0.9998 - val_loss: 0.0861 - val_accuracy: 0.9679
cnn aug  0.834190966266438 percent_data/80/sst2
Epoch 1/20
59/59 [==============================] - 12s 184ms/step - loss: 0.6048 - accuracy: 0.6626 - val_loss: 0.4060 - val_accuracy: 0.8132
Epoch 2/20
59/59 [==============================] - 9s 158ms/step - loss: 0.3533 - accuracy: 0.8461 - val_l

In [4]:
### For training on the original data, without augmentations
#for each method
a_method = 'no_aug' #### TOGGLE EDA/WEDA

writer_cnn = open('outputs_percent/cnn_' + a_method + '_' + get_now_str() + '.txt', 'w')

#for storing the performances
performances_cnn = {size_folder:[] for size_folder in folders_list}

#for each percentage dataset
for size_folder in folders_list:

    writer_cnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]



    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        

        train_path = dataset_folder + '/train_orig.txt' #### TOGGLE EDA/WEDA
        test_path = dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn aug ", acc, dataset_folder)

        performances_cnn[size_folder].append(acc)
        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn[alpha].append(acc)
        print("rnn aug", acc)
        """

writer_cnn.write(str(performances_cnn) + '\n')
#writer_rnn.write(str(performances_rnn)+"\n")
for size_folder in performances_cnn:
    line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
    writer_cnn.write(line + '\n')
    print(line)
"""
for alpha in performances_rnn:
    line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
    writer_rnn.write(line + '\n')
    print(line)
"""
print(performances_cnn)

writer_cnn.close()    


Epoch 1/20
1/1 [==============================] - 1s 861ms/step - loss: 0.6983 - accuracy: 0.4769 - val_loss: 0.6910 - val_accuracy: 0.6250
Epoch 2/20
1/1 [==============================] - 0s 55ms/step - loss: 0.6438 - accuracy: 0.9077 - val_loss: 0.6905 - val_accuracy: 0.6250
Epoch 3/20
1/1 [==============================] - 0s 56ms/step - loss: 0.5933 - accuracy: 0.9692 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 57ms/step - loss: 0.5517 - accuracy: 0.9538 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 58ms/step - loss: 0.5111 - accuracy: 0.9538 - val_loss: 0.6970 - val_accuracy: 0.5000
cnn aug  0.534019439679817 percent_data/1/sst2
Epoch 1/20
1/1 [==============================] - 1s 506ms/step - loss: 0.7147 - accuracy: 0.5000 - val_loss: 0.6831 - val_accuracy: 0.4865
Epoch 2/20
1/1 [==============================] - 0s 99ms/step - loss: 0.6743 - accuracy: 0.5181 - val_loss: 0.6777 -

2/2 [==============================] - 0s 110ms/step - loss: 0.3517 - accuracy: 0.8973 - val_loss: 0.4914 - val_accuracy: 0.7432
Epoch 14/20
2/2 [==============================] - 0s 106ms/step - loss: 0.3217 - accuracy: 0.9054 - val_loss: 0.4799 - val_accuracy: 0.7432
Epoch 15/20
2/2 [==============================] - 0s 115ms/step - loss: 0.2951 - accuracy: 0.9109 - val_loss: 0.4745 - val_accuracy: 0.7432
Epoch 16/20
2/2 [==============================] - 0s 118ms/step - loss: 0.2684 - accuracy: 0.9204 - val_loss: 0.4785 - val_accuracy: 0.7568
Epoch 17/20
2/2 [==============================] - 0s 115ms/step - loss: 0.2448 - accuracy: 0.9263 - val_loss: 0.4635 - val_accuracy: 0.7432
Epoch 18/20
2/2 [==============================] - 0s 121ms/step - loss: 0.2151 - accuracy: 0.9439 - val_loss: 0.4559 - val_accuracy: 0.7703
Epoch 19/20
2/2 [==============================] - 0s 109ms/step - loss: 0.1947 - accuracy: 0.9506 - val_loss: 0.4587 - val_accuracy: 0.7500
Epoch 20/20
2/2 [========

4/4 [==============================] - 1s 162ms/step - loss: 0.6205 - accuracy: 0.6329 - val_loss: 0.5987 - val_accuracy: 0.6802
Epoch 4/20
4/4 [==============================] - 1s 165ms/step - loss: 0.5789 - accuracy: 0.7315 - val_loss: 0.5500 - val_accuracy: 0.7680
Epoch 5/20
4/4 [==============================] - 1s 169ms/step - loss: 0.5325 - accuracy: 0.8042 - val_loss: 0.5096 - val_accuracy: 0.7883
Epoch 6/20
4/4 [==============================] - 1s 160ms/step - loss: 0.4855 - accuracy: 0.8007 - val_loss: 0.4621 - val_accuracy: 0.8243
Epoch 7/20
4/4 [==============================] - 1s 175ms/step - loss: 0.4371 - accuracy: 0.8286 - val_loss: 0.4274 - val_accuracy: 0.8176
Epoch 8/20
4/4 [==============================] - 1s 160ms/step - loss: 0.3937 - accuracy: 0.8460 - val_loss: 0.4086 - val_accuracy: 0.8288
Epoch 9/20
4/4 [==============================] - 1s 165ms/step - loss: 0.3572 - accuracy: 0.8619 - val_loss: 0.3935 - val_accuracy: 0.8243
Epoch 10/20
4/4 [==============

7/7 [==============================] - 1s 150ms/step - loss: 0.3125 - accuracy: 0.8744 - val_loss: 0.3445 - val_accuracy: 0.8351
Epoch 8/20
7/7 [==============================] - 1s 149ms/step - loss: 0.2823 - accuracy: 0.8927 - val_loss: 0.3401 - val_accuracy: 0.8378
Epoch 9/20
7/7 [==============================] - 1s 149ms/step - loss: 0.2477 - accuracy: 0.9186 - val_loss: 0.3586 - val_accuracy: 0.8311
Epoch 10/20
7/7 [==============================] - 1s 151ms/step - loss: 0.2210 - accuracy: 0.9285 - val_loss: 0.3455 - val_accuracy: 0.8324
Epoch 11/20
7/7 [==============================] - 1s 151ms/step - loss: 0.1874 - accuracy: 0.9494 - val_loss: 0.3368 - val_accuracy: 0.8432
Epoch 12/20
7/7 [==============================] - 1s 152ms/step - loss: 0.1551 - accuracy: 0.9652 - val_loss: 0.3398 - val_accuracy: 0.8405
Epoch 13/20
7/7 [==============================] - 1s 148ms/step - loss: 0.1293 - accuracy: 0.9757 - val_loss: 0.3486 - val_accuracy: 0.8419
Epoch 14/20
7/7 [==========